

# WIKIPEDIA HTTP 크롤링



In [1]:
import pandas as pd
tables = pd.read_html("https://ko.wikipedia.org/wiki/HTTP")
len(tables)

13

In [3]:
tables[3]

,HTTP 메소드,RFC,요청에 Body가 있음,응답에 Body가 있음,안전,멱등(Idempotent),캐시 가능
0,GET,RFC 9110,선택 사항,예,예,예,예
1,HEAD,RFC 9110,선택 사항,아니요,예,예,예
2,POST,RFC 9110,예,예,아니요,아니요,예
3,PUT,RFC 9110,예,예,아니요,예,아니요
4,DELETE,RFC 9110,선택 사항,예,아니요,예,아니요
5,CONNECT,RFC 9110,선택 사항,예,아니요,아니요,아니요
6,OPTIONS,RFC 9110,선택 사항,예,예,예,아니요
7,TRACE,RFC 9110,아니요,예,예,예,아니요
8,PATCH,RFC 5789,예,예,아니요,아니요,아니요


# 홈플러스 제품 목록 크롤링

In [4]:
import requests
import pandas as pd
from typing import List, Dict
import time

def fetch_homeplus_data(page: int, per_page: int, promo_no: int, theme_no: int) -> List[Dict]:
    """홈플러스 상품 데이터를 가져오는 함수"""

    base_url = "https://mfront.homeplus.co.kr/promo.json"

    params = {
        "page": page,
        "perPage": per_page,
        "promoNo": promo_no,
        "sort": "NEW",
        "themeNo": theme_no
    }

    # 요청 헤더 설정
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://mfront.homeplus.co.kr/"
    }

    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # HTTP 에러 체크

        data = response.json()
        return data.get("data", {}).get("dataList", [])

    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return []

def create_dataframe(items: List[Dict]) -> pd.DataFrame:
    """상품 데이터를 DataFrame으로 변환하는 함수"""

    # 필요한 컬럼 지정
    columns = [
        'itemNm', 'itemNo', 'brandNm', 'salePrice', 'dcPrice', 'grade',
        'reviewCnt', 'salesCnt', 'soldOutYn', 'lcateNm', 'mcateNm',
        'scateNm', 'dcateNm'
    ]

    # 데이터 추출
    extracted_data = []
    for item in items:
        item_data = {col: item.get(col) for col in columns}
        extracted_data.append(item_data)

    return pd.DataFrame(extracted_data)
# 방법 2: 전역 변수로 설정하고 싶은 경우


def main():

    # 파라미터 설정
    page = 1
    per_page = 20
    promo_no = 13218
    theme_no = 56832

    # 데이터 수집
    items = fetch_homeplus_data(page, per_page, promo_no, theme_no)

    # DataFrame 생성
    df = create_dataframe(items)

    # 결과 출력
    print("수집된 데이터 건수:", len(df))
    print("\n데이터 미리보기:")
    print(df.head())

    # CSV 파일로 저장
    df.to_csv('homeplus_products.csv', index=False, encoding='utf-8-sig')
    print("\n데이터가 'homeplus_products.csv' 파일로 저장되었습니다.")

    return df

In [5]:
main()

수집된 데이터 건수: 18

데이터 미리보기:
               itemNm     itemNo brandNm  salePrice  dcPrice  grade  \
0           당당갈비왕치킨콤보  070882254    None      11990      NaN    4.7   
1    고백스시 특선 초밥 (31입)  070454698    None      21990  20990.0    4.7   
2           솥솥 한판 닭강정  069664773    None      15990  12990.0    4.6   
3          홈플델리 두툼떡갈비  070325270    None      10990   7990.0    4.7   
4  고백스시 실속 모둠초밥 (30입)  070454491    None      17990  14990.0    4.6   

   reviewCnt  salesCnt soldOutYn   lcateNm   mcateNm      scateNm dcateNm  
0        284      7119         N  델리/치킨/초밥  치킨/튀김/구이  후라이드/양념/로스트    후라이드  
1       4761      7275         N  델리/치킨/초밥     초밥/김밥           초밥      초밥  
2       7274      7114         N  델리/치킨/초밥  치킨/튀김/구이  후라이드/양념/로스트      양념  
3       6190     14169         N  델리/치킨/초밥  치킨/튀김/구이         기타요리    기타요리  
4       5040     10344         N  델리/치킨/초밥     초밥/김밥           초밥      초밥  

데이터가 'homeplus_products.csv' 파일로 저장되었습니다.


,itemNm,itemNo,brandNm,salePrice,dcPrice,grade,reviewCnt,salesCnt,soldOutYn,lcateNm,mcateNm,scateNm,dcateNm
0,당당갈비왕치킨콤보,070882254,None,11990,NaN,4.7,284,7119,N,델리/치킨/초밥,치킨/튀김/구이,후라이드/양념/로스트,후라이드
1,고백스시 특선 초밥 (31입),070454698,None,21990,20990.0,4.7,4761,7275,N,델리/치킨/초밥,초밥/김밥,초밥,초밥
2,솥솥 한판 닭강정,069664773,None,15990,12990.0,4.6,7274,7114,N,델리/치킨/초밥,치킨/튀김/구이,후라이드/양념/로스트,양념
3,홈플델리 두툼떡갈비,070325270,None,10990,7990.0,4.7,6190,14169,N,델리/치킨/초밥,치킨/튀김/구이,기타요리,기타요리
4,고백스시 실속 모둠초밥 (30입),070454491,None,17990,14990.0,4.6,5040,10344,N,델리/치킨/초밥,초밥/김밥,초밥,초밥
5,고백스시 특선 초밥 (10입),069464088,None,9990,8990.0,4.7,1071,1028,N,델리/치킨/초밥,초밥/김밥,초밥,초밥
6,대짜 여수 꼬막 비빔밥,070542755,None,12990,10990.0,4.5,1919,2584,N,델리/치킨/초밥,치킨/튀김/구이,기타요리,기타요리
7,홈플델리 갈비왕오븐치킨,070650195,None,10990,7990.0,4.6,1780,3220,N,델리/치킨/초밥,치킨/튀김/구이,후라이드/양념/로스트,로스트
8,고백스시 한판 새우초밥 (20입),070650235,None,17990,15990.0,4.6,365,1242,N,델리/치킨/초밥,초밥/김밥,초밥,초밥
9,홈플델리 매콤직화 제육불고기,070692041,None,7990,7190.0,4.5,369,2387,N,델리/치킨/초밥,치킨/튀김/구이,기타요리,기타요리


# 홈플러스 제품 리뷰 크롤링

In [6]:
import pandas as pd
import requests
import json
from typing import List, Dict, Any

def fetch_reviews(item_no: str, page: int = 1, per_page: int = 30) -> Dict[str, Any]:
    """
    리뷰 데이터를 fetch하는 함수

    Args:
        item_no: 상품 번호
        page: 페이지 번호
        per_page: 페이지당 리뷰 수

    Returns:
        Dict containing the API response
    """
    base_url = "https://mfront.homeplus.co.kr/review/getItemReview.json"

    params = {
        'groupItem': 'N',
        'isAttached': 'false',
        'itemNo': item_no,
        'myStore': 'N',
        'page': page,
        'perPage': per_page,
        'reviewFilter': '',
        'sort': 'reg',
        'storeKind': 'NOR',
        'storeType': 'HYPER',
        'type': 'A'
    }

    response = requests.get(base_url, params=params)
    return response.json()

def process_review_data(reviews: List[Dict[str, Any]]) -> pd.DataFrame:
    """
    리뷰 데이터를 처리하여 데이터프레임으로 변환하는 함수

    Args:
        reviews: List of review dictionaries

    Returns:
        pandas DataFrame containing processed review data
    """
    processed_data = []

    for review in reviews:
        # 기본 리뷰 정보 추출
        review_data = {
            'review_no': review['reviewNo'],
            'user_id': review['userId'],
            'user_name': review['userNm'],
            'grade': review['grade'],
            'grade_name': review['gradeNm'],
            'contents': review['contents'],
            'reg_date': review['regDt'],
            'item_name': review['itemNm'],
            'item_no': review['itemNo'],
            'store_type': review['storeType'],
            'repurchase': review['repurchaseYn'],
            'recommend': review['recommendYn'],
            'like_count': review['likeCnt'],
            'satisfaction_taste': review['satis1AnswerText'],
            'satisfaction_packaging': review['satis2AnswerText']
        }

        # 이미지 정보 처리
        images = review.get('imageList', [])
        review_data['image_count'] = len(images)
        if images:
            review_data['image_urls'] = [img['imgUrl'] for img in images]
        else:
            review_data['image_urls'] = []

        processed_data.append(review_data)

    return pd.DataFrame(processed_data)

def collect_all_reviews(item_no: str, max_pages: int = 10) -> pd.DataFrame:
    """
    모든 페이지의 리뷰를 수집하는 함수

    Args:
        item_no: 상품 번호
        max_pages: 수집할 최대 페이지 수

    Returns:
        pandas DataFrame containing all collected reviews
    """
    all_reviews = []

    for page in range(1, max_pages + 1):
        response_data = fetch_reviews(item_no, page)

        if not response_data['data']:  # 더 이상 데이터가 없으면 중단
            break

        reviews_df = process_review_data(response_data['data'])
        all_reviews.append(reviews_df)

    return pd.concat(all_reviews, ignore_index=True) if all_reviews else pd.DataFrame()

# 사용 예시
if __name__ == "__main__":
    item_no = "069664773"  # 예시 상품 번호
    reviews_df = collect_all_reviews(item_no)

    # 기본 통계 출력
    print(f"총 수집된 리뷰 수: {len(reviews_df)}")
    print("\n평점 분포:")
    print(reviews_df['grade'].value_counts().sort_index())

    # CSV로 저장
    reviews_df.to_csv(f'reviews_{item_no}.csv', index=False, encoding='utf-8-sig')

총 수집된 리뷰 수: 300

평점 분포:
grade
1.0      9
3.0     19
4.0     39
5.0    233
Name: count, dtype: int64


In [9]:
# CSV 파일 로드
df = pd.read_csv('reviews_069664773.csv', encoding='utf-8-sig')
df

,review_no,user_id,user_name,grade,grade_name,contents,reg_date,item_name,item_no,store_type,repurchase,recommend,like_count,satisfaction_taste,satisfaction_packaging,image_count,image_urls
0,4126685561,gof****,이*령,5.0,VIP+,맛있어요 만족합니다.,2025-01-13 20:02:43,솥솥 한판 닭강정,69664773,HYPER,N,N,0,맛있어요,꼼꼼해요,1,['/us/3e415c06-5ce4-40f1-9055-4273d7a37764']
1,4126685078,uj****,엄*섭,5.0,VIP+,용량이나 품질에 만족해요.,2025-01-13 19:52:01,솥솥 한판 닭강정,69664773,HYPER,N,N,0,맛있어요,꼼꼼해요,0,[]
2,4126679608,blue****,김*은,5.0,Gold+,행사해서 저렴하게 구매했어요 \n맛있어서 자주 먹어요,2025-01-13 16:57:41,솥솥 한판 닭강정,69664773,HYPER,Y,N,0,맛있어요,꼼꼼해요,1,['/us/f8cf90ae-1b22-404b-95c5-e417e6025ea8']
3,4126677287,ksh****,강*원,5.0,Silver+,최애쇼핑몰 항상 잘 이용하고 있습니다,2025-01-13 16:04:15,솥솥 한판 닭강정,69664773,HYPER,N,N,0,맛있어요,꼼꼼해요,1,['/us/23b66c1f-9998-4d55-8d48-2b223c902e3c']
4,4126672411,tnwls****,김*진,5.0,VIP+,배민 닭강정보다 훨씬마싰어요,2025-01-13 13:21:49,솥솥 한판 닭강정,69664773,HYPER,Y,N,0,NaN,NaN,1,['/us/8ad6cd64-7ee0-46cb-86b4-9dd3944c6995']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4125936400,awlam****,조*수,4.0,Family+,할인한 가격으로 샀어요,2024-12-22 10:03:31,솥솥 한판 닭강정,69664773,HYPER,N,N,0,맛있어요,꼼꼼해요,1,['/us/1baa0850-d2e6-45bf-973d-2be382058aa6']
296,4125935895,sin****,신*윤,5.0,Silver+,맛도있고 배송도 꼼꼼히 잘 해주셨어요.,2024-12-22 09:39:47,솥솥 한판 닭강정,69664773,HYPER,Y,N,0,맛있어요,꼼꼼해요,0,[]
297,4125935773,$$ho*mebc10b3b9****,양*웅,5.0,Silver+,NaN,2024-12-22 09:35:26,솥솥 한판 닭강정,69664773,HYPER,N,N,0,NaN,NaN,0,[]
298,4125933620,eun****,박*정,4.0,Family+,닭강정 너무 좋아해서 자주구매했는데ㅎ \n솥뚜껑모양이라 재미있어요,2024-12-22 06:19:08,솥솥 한판 닭강정,69664773,HYPER,Y,N,0,맛있어요,꼼꼼해요,1,['/us/112fedb9-7537-41ca-8135-de25537d8ae7']


In [16]:
import pandas as pd
import requests
import json
import time
import sqlite3
from typing import List, Dict, Any
from datetime import datetime

class ReviewCollector:
    def __init__(self, db_name: str = 'reviews.db'):
        """
        리뷰 수집기 초기화

        Args:
            db_name: SQLite 데이터베이스 파일명
        """
        self.db_name = db_name
        self.setup_database()

    def setup_database(self):
        """데이터베이스와 테이블 생성"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS reviews (
                    review_no INTEGER PRIMARY KEY,
                    user_id TEXT,
                    user_name TEXT,
                    grade REAL,
                    grade_name TEXT,
                    contents TEXT,
                    reg_date TEXT,
                    item_name TEXT,
                    item_no TEXT,
                    store_type TEXT,
                    repurchase TEXT,
                    recommend TEXT,
                    like_count INTEGER,
                    satisfaction_taste TEXT,
                    satisfaction_packaging TEXT,
                    image_count INTEGER,
                    image_urls TEXT,
                    collected_at TEXT
                )
            ''')

            # 수집 로그 테이블 생성
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS collection_log (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    item_no TEXT,
                    page INTEGER,
                    records_count INTEGER,
                    collected_at TEXT
                )
            ''')
            conn.commit()

    def fetch_reviews(self, item_no: str, page: int = 1, per_page: int = 30) -> Dict[str, Any]:
        """
        리뷰 데이터를 fetch하는 함수

        Args:
            item_no: 상품 번호
            page: 페이지 번호
            per_page: 페이지당 리뷰 수

        Returns:
            Dict containing the API response
        """
        base_url = "https://mfront.homeplus.co.kr/review/getItemReview.json"

        params = {
            'groupItem': 'N',
            'isAttached': 'false',
            'itemNo': item_no,
            'myStore': 'N',
            'page': page,
            'perPage': per_page,
            'reviewFilter': '',
            'sort': 'reg',
            'storeKind': 'NOR',
            'storeType': 'HYPER',
            'type': 'A'
        }

        response = requests.get(base_url, params=params)
        return response.json()

    def process_review_data(self, reviews: List[Dict[str, Any]]) -> pd.DataFrame:
        """
        리뷰 데이터를 처리하여 데이터프레임으로 변환하는 함수

        Args:
            reviews: List of review dictionaries

        Returns:
            pandas DataFrame containing processed review data
        """
        processed_data = []

        for review in reviews:
            review_data = {
                'review_no': review['reviewNo'],
                'user_id': review['userId'],
                'user_name': review['userNm'],
                'grade': review['grade'],
                'grade_name': review['gradeNm'],
                'contents': review['contents'],
                'reg_date': review['regDt'],
                'item_name': review['itemNm'],
                'item_no': review['itemNo'],
                'store_type': review['storeType'],
                'repurchase': review['repurchaseYn'],
                'recommend': review['recommendYn'],
                'like_count': review['likeCnt'],
                'satisfaction_taste': review['satis1AnswerText'],
                'satisfaction_packaging': review['satis2AnswerText']
            }

            images = review.get('imageList', [])
            review_data['image_count'] = len(images)
            review_data['image_urls'] = json.dumps([img['imgUrl'] for img in images]) if images else '[]'
            review_data['collected_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            processed_data.append(review_data)

        return pd.DataFrame(processed_data)

    def save_to_database(self, df: pd.DataFrame, item_no: str, page: int):
        """
        데이터프레임을 SQLite 데이터베이스에 저장

        Args:
            df: 저장할 데이터프레임
            item_no: 상품 번호
            page: 현재 페이지 번호
        """
        with sqlite3.connect(self.db_name) as conn:
            # 리뷰 데이터 저장
            df.to_sql('reviews', conn, if_exists='append', index=False)

            # 수집 로그 저장
            log_data = pd.DataFrame([{
                'item_no': item_no,
                'page': page,
                'records_count': len(df),
                'collected_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }])
            log_data.to_sql('collection_log', conn, if_exists='append', index=False)

    def collect_reviews(self, item_no: str) -> None:
        """
        모든 페이지의 리뷰를 수집하고 데이터베이스에 저장

        Args:
            item_no: 상품 번호
        """
        page = 1
        total_reviews = 0

        print(f"상품 번호 {item_no}의 리뷰 수집을 시작합니다...")

        while True:
            print(f"\n페이지 {page} 수집 중...")

            try:
                response_data = self.fetch_reviews(item_no, page)

                # if not response_data['data']:  # 더 이상 데이터가 없으면 중단
                    # print("\n더 이상 수집할 데이터가 없습니다.")
                if page == 11:    # 10 page 가 되면 중단
                    print("\n10페이지까지")
                    break

                reviews_df = self.process_review_data(response_data['data'])
                self.save_to_database(reviews_df, item_no, page)

                records_in_page = len(reviews_df)
                total_reviews += records_in_page

                print(f"- 현재 페이지 리뷰 수: {records_in_page}")
                print(f"- 총 수집된 리뷰 수: {total_reviews}")

                time.sleep(0.01)  # 0.01초 대기
                page += 1

            except Exception as e:
                print(f"\n오류 발생: {str(e)}")
                break

        print(f"\n수집 완료!")
        print(f"총 {page-1}개 페이지, {total_reviews}개의 리뷰를 수집했습니다.")

    def get_collection_summary(self):
        """수집된 데이터 요약 정보 조회"""
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()

            # 전체 리뷰 수
            cursor.execute("SELECT COUNT(*) FROM reviews")
            total_reviews = cursor.fetchone()[0]

            # 상품별 리뷰 수
            cursor.execute("""
                SELECT item_no, item_name, COUNT(*) as review_count
                FROM reviews
                GROUP BY item_no, item_name
            """)
            item_summary = cursor.fetchall()

            print("\n=== 데이터 수집 요약 ===")
            print(f"총 리뷰 수: {total_reviews}")
            print("\n상품별 리뷰 수:")
            for item in item_summary:
                print(f"- {item[1]} ({item[0]}): {item[2]}개")

# 사용 예시
if __name__ == "__main__":
    collector = ReviewCollector('homeplus_reviews.db')

    # 리뷰 수집
    item_no = "069664773"
    collector.collect_reviews(item_no)

    # 수집 결과 요약
    collector.get_collection_summary()

상품 번호 069664773의 리뷰 수집을 시작합니다...

페이지 1 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 30

페이지 2 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 60

페이지 3 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 90

페이지 4 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 120

페이지 5 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 150

페이지 6 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 180

페이지 7 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 210

페이지 8 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 240

페이지 9 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 270

페이지 10 수집 중...
- 현재 페이지 리뷰 수: 30
- 총 수집된 리뷰 수: 300

페이지 11 수집 중...

10페이지까지

수집 완료!
총 10개 페이지, 300개의 리뷰를 수집했습니다.

=== 데이터 수집 요약 ===
총 리뷰 수: 300

상품별 리뷰 수:
- 솥솥 한판 닭강정 (069664773): 300개


In [17]:
import pandas as pd
import sqlite3
import json
from typing import Optional, List

class ReviewDataLoader:
    def __init__(self, db_path: str = 'homeplus_reviews.db'):
        """
        리뷰 데이터 로더 초기화

        Args:
            db_path: SQLite 데이터베이스 파일 경로
        """
        self.db_path = db_path

    def load_all_reviews(self) -> pd.DataFrame:
        """모든 리뷰 데이터를 불러옴"""
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql_query("SELECT * FROM reviews", conn)

            # image_urls JSON 문자열을 리스트로 변환
            df['image_urls'] = df['image_urls'].apply(json.loads)

            # 날짜 칼럼을 datetime으로 변환
            df['reg_date'] = pd.to_datetime(df['reg_date'])
            df['collected_at'] = pd.to_datetime(df['collected_at'])

            return df

    def load_reviews_by_item(self, item_no: str) -> pd.DataFrame:
        """특정 상품의 리뷰 데이터만 불러옴"""
        with sqlite3.connect(self.db_path) as conn:
            query = "SELECT * FROM reviews WHERE item_no = ?"
            df = pd.read_sql_query(query, conn, params=[item_no])

            df['image_urls'] = df['image_urls'].apply(json.loads)
            df['reg_date'] = pd.to_datetime(df['reg_date'])
            df['collected_at'] = pd.to_datetime(df['collected_at'])

            return df

    def load_reviews_with_images(self) -> pd.DataFrame:
        """이미지가 있는 리뷰만 불러옴"""
        with sqlite3.connect(self.db_path) as conn:
            query = "SELECT * FROM reviews WHERE image_count > 0"
            df = pd.read_sql_query(query, conn)

            df['image_urls'] = df['image_urls'].apply(json.loads)
            df['reg_date'] = pd.to_datetime(df['reg_date'])
            df['collected_at'] = pd.to_datetime(df['collected_at'])

            return df

    def load_collection_log(self) -> pd.DataFrame:
        """수집 로그 데이터를 불러옴"""
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql_query("SELECT * FROM collection_log", conn)
            df['collected_at'] = pd.to_datetime(df['collected_at'])
            return df

    def get_review_statistics(self) -> dict:
        """리뷰 데이터 기본 통계 정보를 반환"""
        df = self.load_all_reviews()

        stats = {
            'total_reviews': len(df),
            'average_grade': df['grade'].mean(),
            'reviews_with_images': (df['image_count'] > 0).sum(),
            'unique_items': df['item_no'].nunique(),
            'date_range': (df['reg_date'].max() - df['reg_date'].min()).days,
            'grade_distribution': df['grade'].value_counts().to_dict(),
            'repurchase_rate': (df['repurchase'] == 'Y').mean() * 100
        }

        return stats

    def print_data_summary(self):
        """데이터 요약 정보를 출력"""
        stats = self.get_review_statistics()

        print("=== 리뷰 데이터 요약 ===")
        print(f"총 리뷰 수: {stats['total_reviews']:,}개")
        print(f"평균 평점: {stats['average_grade']:.2f}")
        print(f"이미지 포함 리뷰: {stats['reviews_with_images']:,}개")
        print(f"unique 상품 수: {stats['unique_items']:,}개")
        print(f"데이터 기간: {stats['date_range']}일")
        print("\n평점 분포:")
        for grade, count in sorted(stats['grade_distribution'].items()):
            print(f"- {grade}점: {count:,}개")
        print(f"\n재구매 의향 비율: {stats['repurchase_rate']:.1f}%")

# 사용 예시
if __name__ == "__main__":
    loader = ReviewDataLoader('homeplus_reviews.db')

    # 모든 리뷰 데이터 불러오기
    df_all = loader.load_all_reviews()
    print("\n전체 데이터 미리보기:")
    print(df_all.head())

    # 특정 상품의 리뷰만 불러오기
    item_no = "069664773"
    df_item = loader.load_reviews_by_item(item_no)
    print(f"\n상품 {item_no}의 리뷰 수: {len(df_item)}")

    # 이미지가 있는 리뷰만 불러오기
    df_with_images = loader.load_reviews_with_images()
    print(f"\n이미지 포함 리뷰 수: {len(df_with_images)}")

    # 데이터 요약 정보 출력
    loader.print_data_summary()


전체 데이터 미리보기:
    review_no              user_id user_name  grade grade_name  \
0  4125932106  $$ho*me28fd0773****       박*홍    5.0      Gold+   
1  4125933620              eun****       박*정    4.0    Family+   
2  4125935773  $$ho*mebc10b3b9****       양*웅    5.0    Silver+   
3  4125935895              sin****       신*윤    5.0    Silver+   
4  4125936400            awlam****       조*수    4.0    Family+   

                                 contents            reg_date  item_name  \
0                              맛있게 잘먹었습니다 2024-12-22 00:55:59  솥솥 한판 닭강정   
1  닭강정 너무 좋아해서 자주구매했는데ㅎ \n솥뚜껑모양이라  재미있어요  2024-12-22 06:19:08  솥솥 한판 닭강정   
2                                    None 2024-12-22 09:35:26  솥솥 한판 닭강정   
3                  맛도있고 배송도 꼼꼼히 잘 해주셨어요.  2024-12-22 09:39:47  솥솥 한판 닭강정   
4                            할인한 가격으로 샀어요 2024-12-22 10:03:31  솥솥 한판 닭강정   

     item_no store_type repurchase recommend  like_count satisfaction_taste  \
0  069664773      HYPER          Y         N         

In [18]:
# 데이터 로더 인스턴스 생성
loader = ReviewDataLoader('homeplus_reviews.db')

# 전체 데이터 불러오기
df = loader.load_all_reviews()

# 특정 상품의 리뷰만 불러오기
df_item = loader.load_reviews_by_item("069664773")

# 데이터 요약 정보 확인
loader.print_data_summary()

=== 리뷰 데이터 요약 ===
총 리뷰 수: 300개
평균 평점: 4.62
이미지 포함 리뷰: 171개
unique 상품 수: 1개
데이터 기간: 22일

평점 분포:
- 1.0점: 9개
- 3.0점: 19개
- 4.0점: 39개
- 5.0점: 233개

재구매 의향 비율: 37.0%


In [19]:
!pip install konlpy scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 21.4 MB/s eta 0:00:00


In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from konlpy.tag import Okt
import sqlite3
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

class ReviewTextAnalyzer:
    def __init__(self, db_path: str = 'homeplus_reviews.db'):
        """
        리뷰 텍스트 분석기 초기화
        """
        self.db_path = db_path
        self.okt = Okt()
        self.vectorizer = None
        self.lda_model = None

    def load_reviews(self):
        """리뷰 데이터 로드 및 전처리"""
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql_query("SELECT * FROM reviews", conn)
            # None 값을 빈 문자열로 대체
            df['contents'] = df['contents'].fillna('')
            return df

    def preprocess_text(self, text: str) -> str:
        """
        텍스트 전처리
        - None 값 처리
        - 형태소 분석
        - 명사, 형용사 추출
        """
        try:
            # 빈 문자열이나 None 처리
            if not text or pd.isna(text):
                return ''

            # 형태소 분석 및 품사 태깅
            pos_tagged = self.okt.pos(str(text), norm=True, stem=True)

            # 명사와 형용사만 추출 (길이 1 초과)
            words = [word for word, pos in pos_tagged
                    if (pos in ['Noun', 'Adjective'] and len(word) > 1)]

            return ' '.join(words)
        except Exception as e:
            print(f"텍스트 전처리 중 오류 발생: {str(e)}")
            print(f"문제가 된 텍스트: {text}")
            return ''

    def extract_keywords(self, df: pd.DataFrame, n_keywords: int = 20) -> list:
        """단순 빈도 기반 키워드 추출"""
        print("키워드 추출 시작...")

        # 모든 리뷰 텍스트 합치기
        all_words = []
        for text in df['contents']:
            try:
                words = self.preprocess_text(text).split()
                all_words.extend(words)
            except Exception as e:
                print(f"키워드 추출 중 오류 발생: {str(e)}")
                continue

        # 빈도수 계산 및 상위 키워드 추출
        word_counts = Counter(all_words)
        return word_counts.most_common(n_keywords)

    def perform_topic_modeling(self, df: pd.DataFrame, n_topics: int = 5):
        """LDA 토픽 모델링 수행"""
        print("토픽 모델링 시작...")

        try:
            # 텍스트 전처리
            processed_texts = df['contents'].apply(self.preprocess_text)

            # 빈 텍스트 제거
            processed_texts = processed_texts[processed_texts != '']

            if len(processed_texts) == 0:
                print("처리할 텍스트가 없습니다.")
                return []

            # TF-IDF 벡터화
            self.vectorizer = TfidfVectorizer(max_features=1000)
            tfidf_matrix = self.vectorizer.fit_transform(processed_texts)

            # LDA 모델 학습
            self.lda_model = LatentDirichletAllocation(
                n_components=n_topics,
                random_state=42
            )
            self.lda_model.fit(tfidf_matrix)

            # 토픽별 주요 단어 추출
            feature_names = self.vectorizer.get_feature_names_out()
            topics = []
            for topic_idx, topic in enumerate(self.lda_model.components_):
                top_words_idx = topic.argsort()[:-10:-1]
                top_words = [feature_names[i] for i in top_words_idx]
                topics.append({
                    'topic_id': topic_idx,
                    'words': top_words,
                    'weights': topic[top_words_idx].tolist()
                })

            return topics

        except Exception as e:
            print(f"토픽 모델링 중 오류 발생: {str(e)}")
            return []

    def perform_clustering(self, df: pd.DataFrame, n_clusters: int = 5):
        """K-means 군집화 수행"""
        print("군집화 시작...")

        try:
            # 텍스트 전처리
            processed_texts = df['contents'].apply(self.preprocess_text)

            # 빈 텍스트 제거
            mask = processed_texts != ''
            processed_texts = processed_texts[mask]
            df_filtered = df[mask]

            if len(processed_texts) == 0:
                print("처리할 텍스트가 없습니다.")
                return []

            # TF-IDF 벡터화
            if self.vectorizer is None:
                self.vectorizer = TfidfVectorizer(max_features=1000)
            tfidf_matrix = self.vectorizer.fit_transform(processed_texts)

            # 실제 군집 수 조정 (데이터 수가 n_clusters보다 적을 경우)
            n_clusters = min(n_clusters, len(processed_texts))

            # K-means 군집화
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            clusters = kmeans.fit_predict(tfidf_matrix)

            # 군집별 중심점과 가까운 문서 추출
            cluster_reviews = []
            df_filtered = df_filtered.reset_index(drop=True)  # 인덱스 리셋

            for i in range(n_clusters):
                cluster_mask = (clusters == i)
                cluster_docs = tfidf_matrix[cluster_mask]

                if cluster_docs.shape[0] > 0:
                    # 군집 중심점과의 거리 계산
                    centroid = kmeans.cluster_centers_[i]
                    distances = np.linalg.norm(cluster_docs.toarray() - centroid, axis=1)

                    # 중심점과 가장 가까운 문서의 인덱스
                    closest_idx = np.argmin(distances)

                    # 군집의 주요 키워드 추출
                    cluster_terms = np.argsort(centroid)[-10:][::-1]
                    top_terms = [self.vectorizer.get_feature_names_out()[idx]
                               for idx in cluster_terms]

                    # 현재 군집에 속한 리뷰들의 인덱스
                    cluster_indices = np.where(cluster_mask)[0]
                    representative_idx = cluster_indices[closest_idx]

                    cluster_reviews.append({
                        'cluster_id': i,
                        'size': int(cluster_mask.sum()),
                        'keywords': top_terms,
                        'representative_review': df_filtered.iloc[representative_idx]['contents']
                    })

            return cluster_reviews

        except Exception as e:
            print(f"군집화 중 오류 발생: {str(e)}")
            return []

    def analyze_reviews(self):
        """전체 분석 수행"""
        print("리뷰 데이터 분석을 시작합니다...")

        try:
            # 데이터 로드
            df = self.load_reviews()
            print(f"총 {len(df)}개의 리뷰를 분석합니다.\n")

            # 1. 기본 키워드 추출
            print("1. 주요 키워드 추출 중...")
            keywords = self.extract_keywords(df)
            if keywords:
                print("\n[주요 키워드 (빈도수)]")
                for word, count in keywords:
                    print(f"- {word}: {count}회")

            # 2. 토픽 모델링
            print("\n2. 토픽 모델링 수행 중...")
            topics = self.perform_topic_modeling(df)
            if topics:
                print("\n[토픽별 주요 단어]")
                for topic in topics:
                    print(f"\n토픽 {topic['topic_id']+1}:")
                    for word, weight in zip(topic['words'], topic['weights']):
                        print(f"- {word}: {weight:.4f}")

            # 3. 군집화
            print("\n3. 리뷰 군집화 수행 중...")
            clusters = self.perform_clustering(df)
            if clusters:
                print("\n[군집별 특성]")
                for cluster in clusters:
                    print(f"\n군집 {cluster['cluster_id']+1} (크기: {cluster['size']}):")
                    print("주요 키워드:", ', '.join(cluster['keywords']))
                    print("대표 리뷰:", cluster['representative_review'])

        except Exception as e:
            print(f"분석 중 오류 발생: {str(e)}")

# 사용 예시
if __name__ == "__main__":
    analyzer = ReviewTextAnalyzer('homeplus_reviews.db')
    analyzer.analyze_reviews()

리뷰 데이터 분석을 시작합니다...
총 300개의 리뷰를 분석합니다.

1. 주요 키워드 추출 중...
키워드 추출 시작...

[주요 키워드 (빈도수)]
- 맛있다: 120회
- 좋다: 80회
- 닭강정: 39회
- 많다: 28회
- 구매: 27회
- 배송: 25회
- 만족하다: 19회
- 이다: 18회
- 저렴하다: 18회
- 있다: 17회
- 양념: 16회
- 바삭: 14회
- 상품: 14회
- 자주: 13회
- 행사: 13회
- 가격: 12회
- 튀김: 12회
- 같다: 12회
- 포장: 11회
- 가성: 11회

2. 토픽 모델링 수행 중...
토픽 모델링 시작...

[토픽별 주요 단어]

토픽 1:
- 배송: 11.3726
- 닭강정: 7.1250
- 상품: 6.2493
- 맛있다: 5.1862
- 빠르다: 5.0673
- 좋다: 4.4497
- 가성: 4.0657
- 괜찮다: 3.9010
- 솥솥: 3.0633

토픽 2:
- 맛있다: 4.2698
- 행사: 4.1900
- 많다: 3.0008
- 딱딱하다: 2.9849
- 바삭: 2.6614
- 좋아하다: 2.6166
- 달달: 2.3734
- 좋다: 2.2916
- 차갑다: 1.9886

토픽 3:
- 맛있다: 27.4225
- 만족하다: 9.6696
- 양념: 4.4319
- 아이: 4.3672
- 최고: 2.5592
- 물엿: 2.4503
- 넉넉하다: 2.2613
- 많다: 2.1423
- 포장: 1.9981

토픽 4:
- 저렴하다: 10.2283
- 구매: 9.9193
- 자주: 6.5021
- 이다: 4.2394
- 행사: 3.6127
- 일해: 3.1891
- 제품: 3.1333
- 그냥: 2.6130
- 생각: 2.4100

토픽 5:
- 좋다: 18.5570
- 있다: 5.8293
- 할인: 5.7938
- 양도: 4.3291
- 많다: 3.8670
- 맛있다: 3.7226
- 닭강정: 3.3659
- 항상: 3.0764
- 먹기: 2.2845

3. 리뷰 군집화 수행 중...